## Convex Hull for 3D points - **Quick Hull Algorithm**

## Input

In [1]:
import plotly.graph_objects as go # install the necessary libraries incase of error
import numpy as np 
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import random
import timeit
import sys
import os


In [2]:

def readInput(filename):
  try :
    array2D=[]
    with open(filename, 'r') as f:
      for line in f.readlines()[0:]:
        lines=line.rstrip('\n')
        lines=lines.split()
        temp=[]
        for i in lines:
          temp.append(float(i))
        array2D.append(point(temp[0],temp[1],temp[2]))
    return array2D
  except:
  # code for reading Input
    inp = []
    x=[0, 0, 1, 1, 0, 0, 1, 1]
    y=[0, 1, 1, 0, 0, 1, 1, 0]
    z=[0, 0, 0, 0, 1, 1, 1, 1]
    for i in range(0,8):
      inp.append(point(x[i],y[i],z[i]))
  return  inp

In [3]:
def printf(points):
  for i in points:
    print(i.x, i.y, i.z)

## step 1

### 1.1

In [4]:
## Extreme points
def isMin(x,y):
  return x<y
def isMax(x,y):
  return x>y

def extremePoints(points):
  if (len(points) <4):
    print("Invalid number of points")
    sys.exit()
  xMin = xMax = yMin = yMax = zMin = zMax = points[0]
  N = len(points)

  for p in points:
    
    if (xMin.x>p.x):
      xMin = p
    if (xMax.x<p.x):
      xMax = p
    
    if ( p.y<yMin.y):
      yMin = p
    if (p.y>yMax.y):
      yMax = p

    if (p.z<zMin.z ):
      zMin = p
    if ( p.z>zMax.z ):
      zMax = p
    
  ep = [xMin, xMax, yMin, yMax, zMin, zMax]
  return ep


In [5]:
# classes used for storing points, planes
class point:
  def __init__ (self, x,y,z):
    self.x = x
    self.y = y
    self.z = z

  def euclideanDist(self, p2):
    xdist = (self.x - p2.x)**2
    ydist = (self.y - p2.y)**2
    zdist = (self.z - p2.z)**2
    return np.sqrt(xdist + ydist +zdist)

  def dotprod(self,p2):
    return self.x*p2.x + self.y*p2.y + self.z*p2.z

  def crossprod(self,p2):
    x1 = self.y*p2.z - self.z*p2.y
    y1 = -(self.x*p2.z - self.z*p2.x)
    z1 = self.x*p2.y - self.y*p2.x
    return point(x1,y1,z1)

  def normalisedV(self):
    # printf([self])
    mag = self.euclideanDist(point(0,0,0))
    return point(self.x/mag, self.y/mag, self.z/mag)

  def diff(self,p2):
    return point(self.x - p2.x, self.y - p2.y, self.z - p2.z)

  def add(self,p2):
    return point(self.x + p2.x , self.y + p2.y, self.z + p2.z)

  def div(self,x):
    return point(self.x/ x , self.y/x, self.z/x)

  def mul(self,x):
    return point(self.x*x, self.y*x, self.z*x)

In [6]:
# Tetrahedron using the extreme points
def temp_dist(a,p):
  normi=(a[0].diff(a[1])).crossprod(a[1].diff(a[2]))
  normi=normi.normalisedV()
  return abs(normi.dotprod(p.diff(a[0])))
def createSimplex( points):
  # Extreme points
  ep = extremePoints(points)
  # printf(ep)
  # print()
  # for p in ep:
  #   print(p.x, p.y, p.z)

  # 1 - The two extreme points in the ep list [ forms base line for the simplex ]
  extP1 = ep[0]
  extP2 = ep[0]
  for p1 in ep:
    for p2 in ep:
      if ( p1.euclideanDist(p2) > extP1.euclideanDist(extP2)):      
        extP1 = p1
        extP2 = p2
  # 2 - Farthest point from the initial line [ third point needed to form triangle/ plane]
  # Reference - https://onlinemschool.com/math/library/analytic_geometry/p_line/ or  https://math.stackexchange.com/questions/1905533/find-perpendicular-distance-from-point-to-line-in-3d
  distToLine = 0
  extP3 = extP1
  extP4 = extP1
  for pt in points:
    if ( pt != extP1 and pt != extP2):
      d = (extP2.diff(extP1)).div(extP1.euclideanDist(extP2)) 
      t = d.dotprod(pt.diff(extP1)) 
      p = extP1.add( d.mul(t))
      dist = p.euclideanDist(pt)
      if (distToLine < dist):
        distToLine = dist
        extP3 = pt
  # 3 -  Farthest point from the plane [ Fourth point needed to form tetrahedron]
  distToPlane = 0
  for pt in points:
    if (pt!= extP1 and pt!= extP2 and pt!= extP3):
      dist = temp_dist([extP1,extP2,extP3],pt)
      if (dist > distToPlane):
        extP4 = pt
        distToPlane = dist
  internal_pts = [extP1, extP2, extP3, extP4] # internal points
  # visited=(internal_pts)
  return internal_pts

# printf(points)
# print()
# printf(internal_pts)

In [7]:
def find_dist(face,pt):
  n=face.normal
  line=pt.diff(face.p1)
  return n.dotprod(line)

In [8]:
# each facet containes 3 points, and 2d plane, 3 edges
def find_normal(a):
  normi=(a[0].diff(a[1])).crossprod(a[1].diff(a[2]))
  normi=normi.normalisedV()
  for p in internal_pts:
    dist=normi.dotprod(p.diff(a[0]))
    #printf([normi])
    #print(dist)
    if(dist>10**-5):
      normi=normi.mul(-1)
      break
  return normi
class facet:
  def __init__(self, p1, p2, p3):
    self.p1 = p1
    self.p2 = p2
    self.p3 = p3
    self.pts = set()
    self.normal=find_normal([p1,p2,p3])

  def plane_to_point_dist(self,pt):
    v1 = self.plane()
    dist = v1.dotprod(pt.diff(self.p1))
    return dist
  def update(self,lis):
    l=[]
    for i in lis:
      dist=find_dist(self,i)
      if((not i in visited) and (dist>10**-5)):
        l.append(i)
    self.pts=set(l)

## step 2
Updating the facets

In [9]:
visited=[]
visfaces = [] # stores the visible faces from farthest point
edges_list = [] # stores the edges of the horizon, used to add new facets with farthest point
sys.setrecursionlimit(10000000)

def edges(face): # returns edges of the face
  lis = []
  lis.append([face.p1,face.p2])
  lis.append([face.p2,face.p3])
  lis.append([face.p3,face.p1])
  return lis
thres = 10**-9

def checkEdge(e,l): # edges checking
  for i in l:
    if e[0]==i[0] and e[1] == i[1]:
      return 1
    if e[0]==i[1] and e[1] == i[0]:
      return 1
  return 0

def horizon(farPoint,face): # horizon
  dist = find_dist(face,farPoint)
  if (dist < thres):
    return 1
  visfaces.append(face)
  for e in edges(face):
    for neighface in req_faces:
      if (face!= neighface and checkEdge(e,edges(neighface))):
        if  (neighface not in visfaces):
          isHorizon = horizon(farPoint, neighface)
          if (isHorizon):
            edges_list.append(e)
        break
  return 0

In [10]:

def facesUpdate(req_faces):
  while (1):
    check = 0
    for face in req_faces:
      if (len(face.pts)<=0): # Facet is already a convex hull facet
        continue
      else: # when there is a point above the plane
        # farthest point of the facet's point list
        check=1
        lis = list(face.pts)
        pt = lis[0]
        dist_max=find_dist(face,pt)
        for i in lis:
          dist=find_dist(face,i)
          if(dist>dist_max):
            dist_max=dist
            pt=i
        # pt = random.choice(lis)
        # finding the planes that are visible from the farthest point
        visfaces.clear()
        edges_list.clear()
        horizon(pt, face)
        for e in edges_list:
          nFacet = facet(e[0],e[1],pt)
          internalFacetPts = set()
          for f in visfaces:
            internalFacetPts = internalFacetPts.union(f.pts)
          nFacet.update(internalFacetPts)
          req_faces.append(nFacet)
        for f in visfaces:
          req_faces.remove(f)
    if (check==0):
      break

## Main

In [11]:
points=readInput('/content/gourd.txt') ## file name
x1 =[]
y1=[]
z1 =[]
for pt in points:
  x1.append(pt.x)
  y1.append(pt.y)
  z1.append(pt.z)
fig1 = go.Figure(data=[])
fig1.add_scatter3d(x=x1, y=y1, z = z1, mode='markers',
                  marker=dict(size=4,color='red'))
fig1.show()
internal_pts=createSimplex(points)

# Creating facets using internal points
face1=facet(internal_pts[0],internal_pts[1],internal_pts[2])
face2=facet(internal_pts[0],internal_pts[1],internal_pts[3])
face3=facet(internal_pts[0],internal_pts[2],internal_pts[3])
face4=facet(internal_pts[1],internal_pts[2],internal_pts[3])

# adding the outside points to corresponding faces
req_faces=[face1,face3,face2,face4]
face1.update(points)
face2.update(points)
face3.update(points)
face4.update(points)


# Intial tetrahedron display
my3dmesh =[]
for face in req_faces:
  x = [face.p1.x, face.p2.x, face.p3.x]
  y = [face.p1.y, face.p2.y, face.p3.y]
  z = [face.p1.z, face.p2.z, face.p3.z]
  my3dmesh1 = go.Mesh3d(x=x[:3],y=y[:3],z=z[:3], alphahull = -1,opacity=0.6,name='y')
  my3dmesh.append(my3dmesh1)
fig1 = go.Figure(data=[my3dmesh[0],my3dmesh[1],my3dmesh[2],my3dmesh[3]])

x1 =[]
y1=[]
z1 =[]
for pt in points:
  x1.append(pt.x)
  y1.append(pt.y)
  z1.append(pt.z)


fig1.add_scatter3d(x=x1, y=y1, z = z1, mode='markers',
                  marker=dict(size=4,color='red'))
fig1.show()


start = timeit.default_timer()
facesUpdate(req_faces)
print("Time " ,timeit.default_timer() - start)


Time  0.2269086440001047


In [12]:
s=set(req_faces)
cnt=0
points_final=set()
for i in req_faces:
  if(len(i.pts)==0):
    cnt+=1
    points_final.add(i.p1)
    points_final.add(i.p2)
    points_final.add(i.p3)
    # printf([i.p1,i.p2,i.p3,i.normal])
    # print()
print("Number of planes - ",cnt,"Number of vertices -",len(points_final))

Number of planes -  264 Number of vertices - 134


## verifying with convex hull library

In [13]:
array2D=[]
with open('/content/gourd.txt', 'r') as f:
  for line in f.readlines()[0:]:
    lines=line.rstrip('\n')
    lines=lines.split()
    temp=[]
    for i in lines:
      temp.append(float(i))
    array2D.append(temp)
array2D=np.array( array2D)
hull = ConvexHull(array2D)
print("Number of planes - ",cnt,"Number of verices -",len(points_final))

Number of planes -  264 Number of verices - 134


## Plotting the obtained convex hull shape from both cases

In [14]:
data = []
for face in req_faces:
  x = [face.p1.x, face.p2.x, face.p3.x]
  y = [face.p1.y, face.p2.y, face.p3.y]
  z = [face.p1.z, face.p2.z, face.p3.z]
  data1 = go.Mesh3d(x=x[:3],y=y[:3],z=z[:3], alphahull = -1,opacity=0.6,name='y',color='blue')
  data.append(data1)
print("Convex Hull using QuickHull Algorithm")
fig = go.Figure(data=data)


# points
x2 =[]
y2=[]
z2 =[]
for pt in points_final:
  x2.append(pt.x)
  y2.append(pt.y)
  z2.append(pt.z)

x1 =[]
y1=[]
z1 =[]
for pt in points:
  x1.append(pt.x)
  y1.append(pt.y)
  z1.append(pt.z)


fig.add_scatter3d(x=x1, y=y1, z = z1, mode='markers',
                  marker=dict(size=4,color='red'),name="Internal points")
fig.add_scatter3d(x=x2, y=y2, z = z2, mode='markers',
                  marker=dict(size=4,color='green'),name="boundary points")
fig.update_layout(title_text="Convex Hull using QuickHull Algorithm")
fig.show()


my3dmesh = []
for pt in hull.simplices:

  x = [points[pt[0]].x, points[pt[1]].x, points[pt[2]].x]
  y = [points[pt[0]].y, points[pt[1]].y, points[pt[2]].y]
  z = [points[pt[0]].z, points[pt[1]].z, points[pt[2]].z]
  my3dmesh1 = go.Mesh3d(x=x[:3],y=y[:3],z=z[:3], alphahull = -1,opacity=0.6,name='y',color='blue')
  my3dmesh.append(my3dmesh1)


print("Convex Hull [Implemented from inbuilt library] ")
fig1 = go.Figure(data=my3dmesh)
fig1.add_scatter3d(x=x1, y=y1, z = z1, mode='markers',
                  marker=dict(size=4,color='red'),name="Internal points")
fig1.add_scatter3d(x=x2, y=y2, z = z2, mode='markers',
                  marker=dict(size=4,color='green'),name="Boundary points")
fig1.update_layout(title_text="Convex Hull [Implemented from inbuilt library]")

fig1.show()


Convex Hull using QuickHull Algorithm


Convex Hull [Implemented from inbuilt library] 
